In [1]:
import os
# Reemplaza con tu ruta si es diferente
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [2]:
import queries
import pandas as pd
# import ibm_db_dbi
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
conn = ibm_db.connect('DATABASE=' + os.getenv('DATABASE') + ';HOSTNAME=' + os.getenv('HOSTNAME') + ';PORT=' + os.getenv('PORT') + ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + ';PWD=' + os.getenv('PASSWORD_DB') + ';', '', '')
# conn = ibm_db.connect(os.getenv('DATABASE'), os.getenv('USERNAME_DB'), os.getenv('PASSWORD_DB')) # os.getenv('HOSTNAME'), os.getenv('PORT')

if conn:
    print("Conexión exitosa")
else:
    print("Error al conectar")

Conexión exitosa


In [9]:
cod_materia = "MATG1076"

In [5]:
# # stmt_select  = ibm_db.exec_immediate(conn, queries.nota_materia)
# stmt_select  = ibm_db.exec_immediate(conn, queries.get_reprobacion_materias_con_socioeconomico(cod_materia))

In [ ]:
# periodos = [('2020', '2S'), ('2021', '2S'), ('2022', '2S'), ('2023', '2S'), ('2024', '2S'), ('2025', '1S')]
# query = queries.get_reprobacion_materias_con_socioeconomico_custom(cod_materia, periodos_especificos=periodos)

In [7]:
# query

In [10]:
periodos_str = "20201S_20251S"
periodos = queries.crear_periodos_personalizados(2020, '1S', 2025, '1S')
query = queries.get_reprobacion_materias_con_socioeconomico_custom(cod_materia, periodos_especificos=periodos, vez_tomada=[1, 2])

In [11]:
query

"WITH materia_objetivo AS (\n    SELECT cod_estudiante, idperiodo, hd.cod_materia_acad, hd.estado_mat_tomada, vez_tomada, hd.nota1, hd.nota2, promedio, hd.anio, hd.termino\n    FROM espol.historia_anio hd\n    INNER JOIN espol.tbl_periodo_academico pa ON pa.anio=hd.anio AND pa.termino=hd.termino AND pa.estado IN ('E','A') AND pa.tipo='R' AND pa.tipoperiodo='S'\n    WHERE ((hd.anio='2020' AND hd.termino='1S') OR (hd.anio='2020' AND hd.termino='2S') OR (hd.anio='2021' AND hd.termino='1S') OR (hd.anio='2021' AND hd.termino='2S') OR (hd.anio='2022' AND hd.termino='1S') OR (hd.anio='2022' AND hd.termino='2S') OR (hd.anio='2023' AND hd.termino='1S') OR (hd.anio='2023' AND hd.termino='2S') OR (hd.anio='2024' AND hd.termino='1S') OR (hd.anio='2024' AND hd.termino='2S') OR (hd.anio='2025' AND hd.termino='1S'))\n    AND hd.COD_MATERIA_ACAD = 'MATG1076' \n    AND estado_mat_tomada!='AN'\n),\ndatos AS (\n    SELECT ha.cod_estudiante, ha.cod_materia_acad, ha.estado_mat_tomada, ha.nota1, ha.nota2, h

In [12]:
stmt_select  = ibm_db.exec_immediate(conn, query)

In [10]:
# print("Number of affected rows: ", ibm_db.num_rows(stmt_select))

In [11]:
# # Fetch and print the column names
# column_names = []
# while ibm_db.fetch_row(stmt_select) != False:
#     column_name = ibm_db.result(stmt_select, 0) # Get the value from the first column (NAME)
#     column_names.append(column_name)

In [12]:
# tuple = ibm_db.fetch_tuple(stmt_select)
# while tuple != False:
#     print("The COD is : ", tuple[0])
#     print("The año : ", tuple[1])
#     tuple = ibm_db.fetch_tuple(stmt_select)

In [13]:
# Fetch all rows
data_list = []
result = ibm_db.fetch_assoc(stmt_select)
while result:
    # print(result)
    data_list.append(result)
    result = ibm_db.fetch_assoc(stmt_select)

In [14]:
# Close the connection
ibm_db.close(conn)

True

In [15]:
# data_list
# list to dataframe
df = pd.DataFrame(data_list)

In [16]:
df

,COD_ESTUDIANTE,COD_MATERIA_ACAD,ESTADO_MAT_TOMADA,VEZ_TOMADA,NOTA1,NOTA2,ANIO,TERMINO,MAT_TOMADAS,PROM_1PARCIAL,...,METERIALTECHOVIVIENDASEP,METERIALPAREDVIVIENDASEP,VIAACCESOVIVIENDASEP,ABASTECIMIENTOAGUA,RECIBEAYUDA,VALORAYUDA,TIPOBACHILLER,COMIDAS,DISPOSITIVOS,ACCESOINTERNET
0,201912292,MATG1076,RP,1,36,25,2024,2S,67,58,...,None,None,None,None,None,None,None,None,None,None
1,201912292,MATG1076,RP,2,10,0,2025,1S,67,58,...,None,None,None,None,None,None,None,None,None,None
2,201912292,MATG1076,RP,2,10,0,2025,1S,67,58,...,None,None,None,None,None,None,None,None,None,None
3,201912292,MATG1076,RP,1,36,25,2024,2S,67,58,...,None,None,None,None,None,None,None,None,None,None
4,201909447,MATG1076,AP,1,69,77,2021,1S,20,85,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,201911815,MATG1076,AP,1,68,69,2021,1S,20,84,...,None,None,None,None,None,None,None,None,None,None
98,202110276,MATG1076,AP,1,70,64,2024,2S,24,70,...,Zinc /teja /eternit,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachiller Internacional Bilingue,DESAYUNO;REFIGERIO;ALMUERZO;MERIENDA;CENA,PORTÁTIL EXCLUSIVA;CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
99,202200739,MATG1076,AP,1,44,70,2025,1S,26,70,...,Hormigón/losa/cemento,Hormigón / ladrillo / bloque / cemento,Lastrado / calle de tierra,RED PÚBLICA LAS 24 HORAS,NO,"0,00",Bachiller Técnico,DESAYUNO;ALMUERZO;MERIENDA,PORTÁTIL EXCLUSIVA;CELULAR,WIFI EXCLUSIVO DE LA VIVIENDA
100,201801750,MATG1076,AP,1,75,55,2021,2S,34,64,...,Hormigón/losa/cemento,Hormigón / ladrillo / bloque / cemento,Carretera / calle pavimentada o adoquinada / c...,RED PÚBLICA LAS 24 HORAS,NO,"0,00",None,DESAYUNO;ALMUERZO;MERIENDA,CELULAR,CYBER;RED PÚBLICA MUNICIPAL


In [17]:
df.to_csv("data/"+ cod_materia + "_" + periodos_str +  "_reprobacion_materias_con_socioeconomico.csv", index=False)
print("Guardado con exito en", "'data/"+ cod_materia + "_" + periodos_str + "_reprobacion_materias_con_socioeconomico.csv'")

Guardado con exito en 'data/MATG1076_20201S_20251S_reprobacion_materias_con_socioeconomico.csv'
